In [5]:
%matplotlib

Using matplotlib backend: MacOSX


In [38]:
from pathlib import Path
import re
from pprint import pprint

import numpy as np
from scipy import signal, ndimage
import matplotlib.pyplot as plt
from matplotlib import pyplot
import pandas as pd

import emd
import eelbrain
import mne
import trftools

## How to get the IF from the envelope, and turn it into the TRF?

1. Load in the envelope
2. Emd sift the envelope
3. Save the IF value
4. Pack the IF value with the sensor
5. Calculate the IF's TRF

In [3]:
## NATIVES ##

STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
EEG_DIR = DATA_ROOT / 'EEG_Natives' / 'Alice_natives_ICAed_fif'
Native_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d*', path.name)]
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs_Natives'
TRF_DIR.mkdir(exist_ok=True)
print(Native_SUBJECTS)
print(len(Native_SUBJECTS))

['S13_Alice-natives_sfreq-100_raw.fif', 'S14_Alice-natives_sfreq-100_raw.fif', 'S12_Alice-natives_sfreq-100_raw.fif', 'S15_Alice-natives_sfreq-100_raw.fif', 'S11_Alice-natives_sfreq-100_raw.fif', 'S16_Alice-natives_sfreq-100_raw.fif', 'S17_Alice-natives_sfreq-100_raw.fif', 'S18_Alice-natives_sfreq-100_raw.fif', 'S19_Alice-natives_sfreq-100_raw.fif', 'S20_Alice-natives_sfreq-100_raw.fif', 'S21_Alice-natives_sfreq-100_raw.fif', 'S01_Alice-natives_sfreq-100_raw.fif', 'S03_Alice-natives_sfreq-100_raw.fif', 'S04_Alice-natives_sfreq-100_raw.fif', 'S05_Alice-natives_sfreq-100_raw.fif', 'S06_Alice-natives_sfreq-100_raw.fif', 'S08_Alice-natives_sfreq-100_raw.fif', 'S10_Alice-natives_sfreq-100_raw.fif', 'S22_Alice-natives_sfreq-100_raw.fif', 'S25_Alice-natives_sfreq-100_raw.fif', 'S26_Alice-natives_sfreq-100_raw.fif', 'S34_Alice-natives_sfreq-100_raw.fif', 'S35_Alice-natives_sfreq-100_raw.fif', 'S36_Alice-natives_sfreq-100_raw.fif', 'S37_Alice-natives_sfreq-100_raw.fif', 'S38_Alice-natives_sfreq

In [91]:
# Load the broad-band envelope from the gammatone pickle files
envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]  # Load in the data
"""# Code Explanation
envelopeL = []
for stimulus in STIMULI:
    envelope = eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle')
    print(envelope)
    envelopeL.append(envelope)
"""
# To down sample into 100 Hz
envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
#print(envelope)
""" # Code Explanation
envelopeL_2 = []
for x in envelope:
    envelope = x.bin(0.01, dim='time', label='start')
    envelopeL_2.append(envelope)
"""
# To covert it into the NDVar structures
envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
""" # Code Explanation
envelopeL_3 = []
for x in envelope:
    envelope = trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope')
    envelopeL_3.append(envelope)
print(envelopeL_3)
"""
n_envelope = np.concatenate(envelope)

print(n_envelope.shape)
#dir(envelope[1])
#print(envelope[1].get_data())

"""
# Plot every envelope (Don't know why it all overlay into one figs)
for i in range(0, 2):
    p = None
    print(envelope[i].get_data())
    p = plt.plot(envelope[i].get_data())
"""

(73654,)


"\n# Plot every envelope (Don't know why it all overlay into one figs)\nfor i in range(0, 2):\n    p = None\n    print(envelope[i].get_data())\n    p = plt.plot(envelope[i].get_data())\n"

In [89]:
sample_rate = 100
"""  # For IMF & IF testing commands
imf, mask_freqs = emd.sift.mask_sift(envelope[1].get_data(), ret_mask_freq=True, max_imfs=6)  
# delete >>, mask_freqs=30/sample_rate
# xy = envelope data 
#print(mask_freqs * sample_rate)
#emd.plotting.plot_imfs(imf)
IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'nht')

pprint(IF[1:])
print(IF.shape)
#print(IF)
"""
# 

IF_ndarrayLIST = []
for i in range(0, 12):
    imf, mask_freqs = emd.sift.mask_sift(envelope[i].get_data(), ret_mask_freq=True, max_imfs=6)  
    # delete >>, mask_freqs=30/sample_rate   # xy = envelope data 
    print(mask_freqs * sample_rate)
    #emd.plotting.plot_imfs(imf)
    IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') #get the IF
    IF_lenINT = IF.shape[0]
    print(IF.shape) # ==(len(IF.shape[0]), 6))
    IF_ndarrayLIST.append(IF)
    n_IF_ndARRAY = np.concatenate(np.array(IF_ndarrayLIST))   #np.concatenate(np.array(IF_ndarrayLIST)).shape == (73654, 6)
    print(n_IF_ndARRAY.shape)

    

"""
dataDICT = pd.DataFrame(IF, columns = ['IMF1','IMF2','IMF3', 'IMF4','IMF5','IMF6'])
file_name = 'ALL_IF_TRF_predictor_testing_tables.csv'
save_path = DATA_ROOT/ "TRFs_pridictors" / Path(file_name)
dataDICT.to_csv(save_path, sep = "," ,index = False , header = True, encoding = "UTF-8")
"""
"""
[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]>> = 每個IMF最多的頻率
# 每個ＩＭＦ的順時頻率都做一個TRF 去跑統計
4hz>> close to the syllables, 18 hz >> close to phoneme speed, but normally we consider syllables as speed
>> use the third IMF >> 
# IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') get the IF
"""

[21.51142955 10.75571477  5.37785739  2.68892869  1.34446435  0.67223217]
(5862, 6)
(5862, 6)
[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]
(6193, 6)
(12055, 6)


/var/folders/h7/sdf5cvgx3qzf5rh744693x0r0000gn/T/ipykernel_69032/3230898397.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  n_IF_ndARRAY = np.concatenate(np.array(IF_ndarrayLIST))   #np.concatenate(np.array(IF_ndarrayLIST)).shape == (73654, 6)


[23.70220703 11.85110351  5.92555176  2.96277588  1.48138794  0.74069397]
(6434, 6)
(18489, 6)
[22.66779232 11.33389616  5.66694808  2.83347404  1.41673702  0.70836851]
(7107, 6)
(25596, 6)
[23.21852732 11.60926366  5.80463183  2.90231591  1.45115796  0.72557898]
(6736, 6)
(32332, 6)
[22.4525975  11.22629875  5.61314938  2.80657469  1.40328734  0.70164367]
(6487, 6)
(38819, 6)
[22.8508909  11.42544545  5.71272273  2.85636136  1.42818068  0.71409034]
(6398, 6)
(45217, 6)
[21.77598904 10.88799452  5.44399726  2.72199863  1.36099931  0.68049966]
(5839, 6)
(51056, 6)
[21.08557709 10.54278854  5.27139427  2.63569714  1.31784857  0.65892428]
(5831, 6)
(56887, 6)
[21.98877306 10.99438653  5.49719326  2.74859663  1.37429832  0.68714916]
(6235, 6)
(63122, 6)
[21.72925764 10.86462882  5.43231441  2.71615721  1.3580786   0.6790393 ]
(5725, 6)
(68847, 6)
[21.67672145 10.83836072  5.41918036  2.70959018  1.35479509  0.67739755]
(4807, 6)
(73654, 6)


"\n[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]>> = 每個IMF最多的頻率\n# 每個ＩＭＦ的順時頻率都做一個TRF 去跑統計\n4hz>> close to the syllables, 18 hz >> close to phoneme speed, but normally we consider syllables as speed\n>> use the third IMF >> \n# IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') get the IF\n"

In [41]:
# To save the IF into a csv file
dataDICT = pd.DataFrame(IF, columns = ['IMF1','IMF2','IMF3', 'IMF4','IMF5','IMF6'])
#data_path = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results/TRFs_pridictors")
file_name = 'IF_TRF_predictor_testing_tables.csv'
save_path = DATA_ROOT/ "TRFs_pridictors" / Path(file_name)
dataDICT.to_csv(save_path, sep = "," ,index = False , header = True, encoding = "UTF-8")

In [94]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-

"""This script estimates TRFs for several models and saves them"""
from pathlib import Path
import re

import eelbrain
import mne
import trftools

if __name__ == "__main__":
    
    STIMULI = [str(i) for i in range(1, 13)]
    DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")#Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
    PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
    PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
    EEG_DIR = DATA_ROOT / 'EEG_Natives' / 'Alice_natives_ICAed_fif'
    SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d*', path.name[:4])]
    # Define a target directory for TRF estimates and make sure the directory is created
    TRF_DIR = DATA_ROOT / 'TRFs_Natives'
    TRF_DIR.mkdir(exist_ok=True)
    print(SUBJECTS)
    print(len(SUBJECTS))
    
    # Load stimuli
    # ------------
    # Make sure to name the stimuli so that the TRFs can later be distinguished
    # Load the gammatone-spectrograms; use the time axis of these as reference
    gammatone = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-8.pickle') for stimulus in STIMULI]
    
    # Resample the spectrograms to 100 Hz (time-step = 0.01 s), which we will use for TRFs
    gammatone = [x.bin(0.01, dim='time', label='start') for x in gammatone]
    
    # Pad onset with 100 ms and offset with 1 second; make sure to give the predictor a unique name as that will make it easier to identify the TRF later
    gammatone = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='gammatone') for x in gammatone]
    #"""
    # Load the broad-band envelope and process it in the same way
    envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]  # Load in the data
    envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
    envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
    onset_envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-on-1.pickle') for stimulus in STIMULI]
    onset_envelope = [x.bin(0.01, dim='time', label='start') for x in onset_envelope]
    onset_envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='onset') for x in onset_envelope]
    
    # Load onset spectrograms and make sure the time dimension is equal to the gammatone spectrograms
    gammatone_onsets = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-on-8.pickle') for stimulus in STIMULI]
    gammatone_onsets = [x.bin(0.01, dim='time', label='start') for x in gammatone_onsets]
    gammatone_onsets = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_onsets, gammatone)]
    #"""
    # Load word tables and convert tables into continuous time-series with matching time dimension
    word_tables = [eelbrain.load.unpickle(PREDICTOR_word_DIR / f'{stimulus}~Ngram-CFG_word.pickle') for stimulus in STIMULI]
    word_onsets = [eelbrain.event_impulse_predictor(gt.time, ds=ds, name='word') for gt, ds in zip(gammatone, word_tables)] # not sure why they could get the word onset this way
    
    # Function and content word impulses based on the boolean variables in the word-tables
    word_lexical = [eelbrain.event_impulse_predictor(gt.time, value='lexical', ds=ds, name='lexical') for gt, ds in zip(gammatone, word_tables)]
    word_nlexical = [eelbrain.event_impulse_predictor(gt.time, value='nlexical', ds=ds, name='non_lexical') for gt, ds in zip(gammatone, word_tables)]
    
    # NGRAM/CFG word impulses based on the values in the word-tables
    word_Ngram = [eelbrain.event_impulse_predictor(gt.time, value='NGRAM', ds=ds, name='n-gram') for gt, ds in zip(gammatone, word_tables)]
    word_CFG = [eelbrain.event_impulse_predictor(gt.time, value='CFG', ds=ds, name='cfg') for gt, ds in zip(gammatone, word_tables)]
    
    # Extract the duration of the stimuli, so we can later match the EEG to the stimuli
    durations = [gt.time.tmax for stimulus, gt in zip(STIMULI, gammatone)]
    
    # Models
    # ------
    # Pre-define models here to have easier access during estimation. In the future, additional models could be added here and the script re-run to generate additional TRFs.
    models = {
        # Acoustic models
        'envelope': [envelope],
        'envelope+onset': [envelope, onset_envelope],
        'acoustic': [gammatone, gammatone_onsets],
        
        # Models with word-onsets and word-class
        'words': [word_onsets],
        'words+lexical': [word_onsets, word_lexical, word_nlexical],
        'acoustic+words': [gammatone, gammatone_onsets, word_onsets],
        'acoustic+words+lexical': [gammatone, gammatone_onsets, word_onsets, word_lexical, word_nlexical],

        # Language Models
        'Ngram': [word_Ngram, word_onsets, word_lexical, word_nlexical],
        'CFG': [word_CFG, word_onsets, word_lexical, word_nlexical],
        'Ngram-CFG_all': [word_Ngram, word_CFG, word_onsets, word_lexical, word_nlexical]

    }
    """
    # Acoustic models
    'envelope': [envelope],
    'envelope+onset': [envelope, onset_envelope],
    'acoustic': [gammatone, gammatone_onsets],
    # Models with word-onsets and word-class
    'words': [word_onsets],
    'words+lexical': [word_onsets, word_lexical, word_nlexical],
    'acoustic+words': [gammatone, gammatone_onsets, word_onsets],
    'acoustic+words+lexical': [gammatone, gammatone_onsets, word_onsets, word_lexical, word_nlexical],
    # Language Models
    'Ngram': [word_Ngram, word_onsets, word_lexical, word_nlexical],
    'CFG': [word_CFG, word_onsets, word_lexical, word_nlexical],
    'Ngram-CFG_all': [word_Ngram, word_CFG, word_onsets, word_lexical, word_nlexical]
    """    
    
    print(envelope)
    # Estimate TRFs
    # -------------
    # Loop through subjects to estimate TRFs
    for subject in SUBJECTS:
        subject_trf_dir = TRF_DIR / subject[:3]
        subject_trf_dir.mkdir(exist_ok=True)
        # Generate all TRF paths so we can check whether any new TRFs need to be estimated
        trf_paths = {model: subject_trf_dir / f'{subject[:3]} {model}.pickle' for model in models}
        # Skip this subject if all files already exist
        #if all(path.exists() for path in trf_paths.values()):
            #continue
        # Load the EEG data
        raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)  # subject /
        # Band-pass filter the raw data between 0.5 and 20 Hz
        raw.filter(0.5, 20)
        # Interpolate bad channels
        raw.interpolate_bads()
        # Extract the events marking the stimulus presentation from the EEG file
        events = eelbrain.load.fiff.events(raw)
        # Not all subjects have all trials; determine which stimuli are present
        trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event']]
        # Extract the EEG data segments corresponding to the stimuli
        trial_durations = [durations[i] for i in trial_indexes]
        eeg = eelbrain.load.fiff.variable_length_epochs(events, -0.100, trial_durations, connectivity='auto')  #, decim=5 #decim=5 meaning to resample to sfreq=100Hz
        # Since trials are of unequal length, we will concatenate them for the TRF estimation.
        eeg_concatenated = eelbrain.concatenate(eeg)
        


['S13_Alice-natives_sfreq-100_raw.fif', 'S14_Alice-natives_sfreq-100_raw.fif', 'S12_Alice-natives_sfreq-100_raw.fif', 'S15_Alice-natives_sfreq-100_raw.fif', 'S11_Alice-natives_sfreq-100_raw.fif', 'S16_Alice-natives_sfreq-100_raw.fif', 'S17_Alice-natives_sfreq-100_raw.fif', 'S18_Alice-natives_sfreq-100_raw.fif', 'S19_Alice-natives_sfreq-100_raw.fif', 'S20_Alice-natives_sfreq-100_raw.fif', 'S21_Alice-natives_sfreq-100_raw.fif', 'S01_Alice-natives_sfreq-100_raw.fif', 'S03_Alice-natives_sfreq-100_raw.fif', 'S04_Alice-natives_sfreq-100_raw.fif', 'S05_Alice-natives_sfreq-100_raw.fif', 'S06_Alice-natives_sfreq-100_raw.fif', 'S08_Alice-natives_sfreq-100_raw.fif', 'S10_Alice-natives_sfreq-100_raw.fif', 'S22_Alice-natives_sfreq-100_raw.fif', 'S25_Alice-natives_sfreq-100_raw.fif', 'S26_Alice-natives_sfreq-100_raw.fif', 'S34_Alice-natives_sfreq-100_raw.fif', 'S35_Alice-natives_sfreq-100_raw.fif', 'S36_Alice-natives_sfreq-100_raw.fif', 'S37_Alice-natives_sfreq-100_raw.fif', 'S38_Alice-natives_sfreq

/var/folders/h7/sdf5cvgx3qzf5rh744693x0r0000gn/T/ipykernel_69032/3354584463.py:120: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()
